### Тема: Сравнение характеристик распределения двух генеральных совокупностей параметрическими методами

**Выполнил**: Лежнин Максим Витальевич (ПМ-31)

**Преподаватель**: Олейник Т.А., кафедра ВМ-1

###### Практическая работа № **8**, вариант № **3**

###### Осенний семестр, 2023 год

###### МИЭТ, Зеленоград

### Библиотеки

In [1]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn

### Задание 1
Сгенерируйте две выборки (A и B) различных объемов из нормально распределенных генеральных совокупностей с параметрами $m_A,\, \sigma_A$ и $m_B,\, \sigma_B$ (возьмите значения $m_A$ и $m_B$ близкими, но не равными, а $\sigma_A$ и $\sigma_B$ выберите такими, чтобы близки (но не равны) были их квадраты). Проверьте гипотезу о равенстве дисперсий по критерию Фишера (см. п. 2 подраздела 8.2) на уровне значимости 0,05. Сформулируйте выводы.

Если гипотеза подтвердится, то сгенерируйте две другие выборки (C и D), для которых гипотеза отклонится (при необходимости поварьируйте параметры, поэкспериментируйте, чтобы добиться заданного результата).

Если гипотеза о равенстве дисперсий выборок A и B будет отклонена, то сгенерируйте две другие выборки (C' и D'), для которых гипотеза подтвердится (при необходимости поварьируйте параметры, поэкспериментируйте, чтобы добиться заданного результата).

In [2]:
# тест Фишера на равенство дисперсий
def f_test_for_var(sample_1, sample_2):
    n_1 = len(sample_1)
    n_2 = len(sample_2)
    
    S_1 = n_1 / (n_1 - 1) * np.var(sample_1)
    S_2 = n_2 / (n_2 - 1) * np.var(sample_2)

    if S_1 > S_2:
        ssv = S_1 / S_2
        pvalue = 2 * min(sts.f(n_1 - 1, n_2 - 1).cdf(ssv), 
              1 - sts.f(n_1 - 1, n_2 - 1).cdf(ssv))
    else:
        ssv = S_2 / S_1
        pvalue = 2 * min(sts.f(n_2 - 1, n_1 - 1).cdf(ssv), 
              1 - sts.f(n_2 - 1, n_1 - 1).cdf(ssv))
    return (ssv, pvalue)

# красивый вывод результатов на экран
def print_results(alpha, sample_1, sample_2):
    test_results = f_test_for_var(sample_1, sample_2)

    print(f"Выборочная дисперсия первой выборки: {np.var(sample_1)}")
    print(f"Выборочная дисперсия второй выборки: {np.var(sample_2)}")
    print(f"Статистика: {test_results[0]}")
    print(f"Значение p: {test_results[1]}")
    print()

    if alpha < test_results[1]:
        print("Гипотеза принимается!")
    else:
        print("Гипотеза отклоняется!")

In [3]:
# характеристики для первой выборки
m_A = 5
sig_A = 3
sample_size_A = 1000

# характеристики для второй выборки
m_B = m_A + 0.2
sig_B = np.sqrt(sig_A**2 + 0.5)
sample_size_B = 2000

# уровень значимости
alpha = 0.05

# наши выборки
A = np.random.normal(m_A, sig_A, sample_size_A)
B = np.random.normal(m_B, sig_B, sample_size_B)

# вывод результатов на экран
print_results(alpha, A, B)

Выборочная дисперсия первой выборки: 9.313266907748163
Выборочная дисперсия второй выборки: 9.370428350082632
Статистика: 1.0056343155307612
Значение p: 0.9232081260793084

Гипотеза принимается!


In [4]:
# гипотеза была принята, потому надо еще чуть увеличить
# среднеквадратичное отклонение
sig_B = round(np.sqrt(sig_A**2 + 0.5)) + 0.1

# получаем новые выборки
C = np.random.normal(m_A, sig_A, sample_size_A)
D = np.random.normal(m_B, sig_B, sample_size_B)

# вывод результатов на экран
print_results(alpha, C, D)

Выборочная дисперсия первой выборки: 8.46850400201933
Выборочная дисперсия второй выборки: 9.983010966869708
Статистика: 1.178250250812393
Значение p: 0.003092173128787401

Гипотеза отклоняется!


### Задание 2
Для выполнения задания используйте пару выборок, смоделированных при выполнении задания 1, для которой гипотеза о
равенстве дисперсий подтвердилась. Проверьте гипотезу о равенстве математических ожиданий. Укажите диапазон значений $\alpha$, при которых гипотеза принимается.

In [5]:
# критерий на равенство мат ожиданий при положительном результате F-теста
def mean_test_if_ftest(sample_1, sample_2):
    n_1 = len(sample_1)
    n_2 = len(sample_2)
    
    S_1 = n_1 / (n_1 - 1) * np.var(sample_1)
    S_2 = n_2 / (n_2 - 1) * np.var(sample_2)
    
    ssv = (np.mean(sample_1) - np.mean(sample_2)) / np.sqrt((((n_1 - 1) * S_1 + 
                    (n_2 - 1) * S_2) / (n_1 + n_2 - 2)) * (1 / n_1 + 1 / n_2))

    pvalue = 2 * min(sts.t(n_1 + n_2 - 2).cdf(ssv), 1 - sts.t(n_1 + n_2 - 2).cdf(ssv))
    
    return (ssv, pvalue)

# красивый вывод на экран
def printing_results(sample_1, sample_2, test):
    test_results = test(sample_1, sample_2)

    print(f"Выборочное мат ожидание первой выборки: {np.mean(sample_1)}")
    print(f"Выборочное мат ожидание второй выборки: {np.mean(sample_2)}")
    print(f"Статистика: {test_results[0]}")
    print(f"Значение p: {test_results[1]}")
    print()

    print(f"Гипотеза принимается при α < {test_results[1]}")

In [6]:
# полезный комментарий
printing_results(A, B, mean_test_if_ftest)

Выборочное мат ожидание первой выборки: 5.109669268807151
Выборочное мат ожидание второй выборки: 5.185719473688975
Статистика: -0.6419071601281596
Значение p: 0.520982540919311

Гипотеза принимается при α < 0.520982540919311


### Задание 3
Для выполнения этого задания используйте пару выборок, смоделированных при выполнении задания 1, для которой гипотеза
о равенстве дисперсий была отклонена. Проверьте гипотезу о равенстве математических ожиданий. Укажите диапазон значений $\alpha$, при которых гипотеза принимается.

In [7]:
# критерий на равенство мат ожиданий при отрицательном результате F-теста
def welch_t_test(sample_1, sample_2):
    n_1 = len(sample_1)
    n_2 = len(sample_2)
    
    S_1 = n_1 / (n_1 - 1) * np.var(sample_1)
    S_2 = n_2 / (n_2 - 1) * np.var(sample_2)

    ssv = (np.mean(sample_1) - np.mean(sample_2)) / np.sqrt(S_1 / n_1 + S_2 / n_2)

    k = (S_1 / n_1 + S_2 / n_2)**2 / ((S_1 / n_1)**2 / (n_1 - 1) + (S_2 / n_2)**2 / 
                                                                          (n_2 - 1))

    pvalue = 2 * min(sts.t(k).cdf(ssv), 1 - sts.t(k).cdf(ssv))

    return (ssv, pvalue)

In [8]:
# полезный комментарий
printing_results(C, D, welch_t_test)

Выборочное мат ожидание первой выборки: 4.973975187313621
Выборочное мат ожидание второй выборки: 5.129589348755445
Статистика: -1.3407551234665718
Значение p: 0.180141615440855

Гипотеза принимается при α < 0.180141615440855


### Задание 4
В файлах «Данные 8_1» приведены массивы результатов 15 тестов, пройденных студентами двух потоков. Будем интерпретировать результаты, относящиеся к разным потокам, как две серии испытаний. Выберите в соответствии с вашим вариантом (варианты распределяются преподавателем) результаты выполнения одного из тестов. Интересующее нас событие M: студент успешно выполнил тест (получил не менее 67 % от максимального балла). Проверьте гипотезу об однородности выборок (равенстве вероятностей события М в двух потоках) (см. п. 5 подраздела 8.2).

In [9]:
# конвертирую лист строк в лист float'ов
def parse_sample(sample, ignoring=0):
    new_sample = []
    for i in sample:
        if i == '-':
            continue
    
        elif i == 0 and ignoring:
            continue

        new_sample.append(float(i))

    return new_sample

# критерий (второй) для проверки равенства выероятностей событий
def test_for_equal_prob(sample_1, sample_2):
    n_1 = len(sample_1)
    n_2 = len(sample_2)
    n = n_1 + n_2
    
    k_1 = sum([int(i >= 1) for i in sample_1])
    k_2 = sum([int(i >= 1) for i in sample_2])
    
    m = min(((k_1 + k_2) * n_1 / n), ((k_1 + k_2) * n_2 / n), 
            (n - k_1 - k_2) * n_1 / n, (n - k_1 - k_2) * n_2 / n)

    if m > 3:
        ssv = n / (n_1 * n_2 * k_1 * k_2)
        pvalue = 2 * min(sts.chi2(1).cdf(ssv), 1 - sts.chi2(1).cdf(ssv))

    else:
        return None

    return (ssv, pvalue)

# красивый вывод на экран
def printing_results(sample_1, sample_2):
    test_results = test_for_equal_prob(sample_1, sample_2)

    n_1 = len(sample_1)
    n_2 = len(sample_2)
    k_1 = sum([int(i >= 1) for i in sample_1])
    k_2 = sum([int(i >= 1) for i in sample_2])
    
    print(f"Выборочная частота события M первой выборки: {k_1 / n_1}")
    print(f"Выборочная частота события M второй выборки: {k_2 / n_2}")
    print(f"Статистика: {test_results[0]}")
    print(f"Значение p: {test_results[1]}")
    print()

    print(f"Гипотеза принимается при α < {test_results[1]}")

In [10]:
# номер варианта
var = 3

# обрабатывая данные
data = pd.read_excel('Data_8_1.ods')
table = pd.DataFrame(data)

# получаю выборки
sample_1 = parse_sample(table[table.columns[var]].dropna())
sample_2 = parse_sample(table[table.columns[var + 17]].dropna())

# вывожу результаты на экран
printing_results(sample_1, sample_2)

Выборочная частота события M первой выборки: 0.7835051546391752
Выборочная частота события M второй выборки: 0.7168141592920354
Статистика: 3.1122215517607795e-06
Значение p: 0.0028151729754487395

Гипотеза принимается при α < 0.0028151729754487395
